In [1]:
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras import regularizers

In [2]:
(train_data, train_label), (test_data, test_label) = imdb.load_data(num_words=10000)

In [3]:
word_index = imdb.get_word_index()
reverse_word_index = dict(
    [(value, key) for (key, value) in word_index.items()]
    )
decoded_review = ' '.join(
    [reverse_word_index.get(i-3, '?') for i in train_data[0]]
)

In [4]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [5]:
def build_model():
    model = models.Sequential()
    model.add(
        layers.Dense(16, activation='relu', input_shape=(10000,), kernel_regularizer=regularizers.l2(0.001))
    )
    model.add(
        layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.001))
    )
    model.add(
        layers.Dense(1, activation='sigmoid')
    )
    return model

In [6]:
def build_model():
    model = models.Sequential()
    model.add(
        layers.Dense(16, activation='relu', input_shape=(10000,))
    )
    model.add(
        layers.Dense(16, activation='relu')
    )
    model.add(
        layers.Dense(1, activation='sigmoid')
    )
    return model

In [7]:
from tools.K_split import K_data_split
from sklearn.model_selection import train_test_split
import datetime

In [8]:
k = 3
val_num_samples = len(x_train)
for i in range(k):
    log_dir = '/home/oneran/Mycodes/DeepLearning/log/imdb'+str(datetime.datetime.now())
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    print('Process---'+str(i))
    train_data, train_labels, val_data, val_labels = K_data_split(k, x_train, train_label, i)
    model = build_model()
    model.compile(
        optimizer='adagrad',
        metrics=['accuracy'],
        loss='binary_crossentropy'
    )
    history = model.fit(
        train_data, train_labels,
        epochs=100,
        batch_size=512,
        validation_data=(val_data, val_labels),
        callbacks=tensorboard_callback
    )

Process---0
Epoch 1/100
33/33 [==============================] - 3s 69ms/step - loss: 0.6940 - accuracy: 0.5032 - val_loss: 0.6921 - val_accuracy: 0.5146
Epoch 2/100
33/33 [==============================] - 1s 31ms/step - loss: 0.6919 - accuracy: 0.5259 - val_loss: 0.6907 - val_accuracy: 0.5298
Epoch 3/100
33/33 [==============================] - 1s 31ms/step - loss: 0.6902 - accuracy: 0.5511 - val_loss: 0.6896 - val_accuracy: 0.5421
Epoch 4/100
33/33 [==============================] - 1s 31ms/step - loss: 0.6890 - accuracy: 0.5618 - val_loss: 0.6884 - val_accuracy: 0.5502
Epoch 5/100
33/33 [==============================] - 1s 27ms/step - loss: 0.6874 - accuracy: 0.5765 - val_loss: 0.6870 - val_accuracy: 0.5639
Epoch 6/100
33/33 [==============================] - 1s 31ms/step - loss: 0.6855 - accuracy: 0.5897 - val_loss: 0.6852 - val_accuracy: 0.5751
Epoch 7/100
33/33 [==============================] - 1s 27ms/step - loss: 0.6838 - accuracy: 0.5983 - val_loss: 0.6829 - val_accuracy: 0

In [19]:
model.evaluate(x_test, test_label)

782/782 [==============================] - 1s 2ms/step - loss: 0.3758 - accuracy: 0.8470


[0.37584277987480164, 0.847000002861023]

In [20]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 512)               5120512   
_________________________________________________________________
dense_22 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 513       
Total params: 5,383,681
Trainable params: 5,383,681
Non-trainable params: 0
_________________________________________________________________
